In [35]:
import pandas as pd
import glob
import subprocess
import os
import time
import psutil
import time
from typing import NamedTuple

## Notebook Utils

In [36]:
METADATA = pd.read_csv('metadata.tsv', sep="\t", index_col="Analyte ID")


In [37]:
class TVCResult(NamedTuple):
    """Container for storing TVC  results."""
    sample_id: str
    runtime_minutes: float
    memory_mb: float
    cores_used: int
    vcf_path: str

In [38]:
class BenchmarkResult(NamedTuple):
    """Container for storing benchmarking results."""
    sample_id: str
    snp_precision: float
    snp_recall: float
    snp_f1: float
    indel_precision: float
    indel_recall: float
    indel_f1: float

In [39]:
class SampleMetadata(NamedTuple):
    """Container for storing sample metadata."""
    sample_id: str
    median_depth_of_coverage: int
    kit: str

In [40]:

# VCF and Benchmark regions from https://www.nist.gov/programs-projects/genome-bottle version v4.2.1
BENCHMARK_VCF = "HG001_GRCh38_1_22_v4.2.1_benchmark.vcf.gz"
BENCHMARK_REGIONS = "HG001_GRCh38_1_22_v4.2.1_benchmark.bed"
REFERENCE_GENOME = "hg38.fa"

In [41]:
def run_TVC(input_bam, input_ref, cores):
    """
    Run TVC variant caller and monitor resource usage across all threads/processes.
    
    :param input_bam: Path to input BAM file.
    :param input_ref: Path to reference genome file.
    :param cores: Number of CPU cores to use.
    :return: TVCResult containing runtime and memory usage information.
    """
    output_vcf = input_bam.replace(".bam", ".tvc.vcf")
    cmd = f"target/release/taps_variant_caller -t {cores} {input_ref} {input_bam} {output_vcf}"

    start_time = time.time()
    proc = subprocess.Popen(cmd, shell=True)
    ps_proc = psutil.Process(proc.pid)

    peak_memory = 0
    while proc.poll() is None:
        try:
            # Include child processes’ memory
            mem_total = ps_proc.memory_info().rss
            for child in ps_proc.children(recursive=True):
                mem_total += child.memory_info().rss
            peak_memory = max(peak_memory, mem_total)
        except psutil.NoSuchProcess:
            break
        time.sleep(0.05)

    end_time = time.time()
    elapsed_minutes = (end_time - start_time) / 60

    result = TVCResult(
        sample_id=input_bam.split(".")[0],
        runtime_minutes=elapsed_minutes,
        memory_mb=peak_memory / (1024 ** 2),
        vcf_path=output_vcf,
        cores_used=cores
    )
    return result


In [42]:
def extract_accuracy_metrics(benchmark_vcf, test_vcf, regions_bed, reference_genome):
    """
    Extract accuracy metrics using hap.py Docker container.
    :param benchmark_vcf: Path to benchmark VCF file.
    :param test_vcf: Path to test VCF file.
    :param regions_bed: Path to BED file defining regions of interest.
    :param reference_genome: Path to reference genome file.
    :return: BenchmarkResult containing precision, recall, and F1 scores for SNPs and INDELs.
    """
    random_suffix = str(os.getpid())
    docker_image = "quay.io/biocontainers/hap.py:0.3.14--py27h5c5a3ab_0"
    cmd = f"docker run --rm -v {os.getcwd()}:/data {docker_image} hap.py /data/{benchmark_vcf} /data/{test_vcf} -r /data/{reference_genome} -f /data/{regions_bed} -o /data/temp_happy_output.{random_suffix} --threads 90"
    subprocess.run(cmd, shell=True, stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
    metrics_file = f'temp_happy_output.{random_suffix}.summary.csv'

    df = pd.read_csv(metrics_file)
    for row in df.itertuples():
        if row.Filter != 'PASS':
            continue
        if row.Type == 'SNP':
            snp_precision = row._12
            snp_recall = row._11
            snp_f1 = row._14
        elif row.Type == 'INDEL':
            indel_precision = row._12
            indel_recall = row._11
            indel_f1 = row._14

    os.remove(metrics_file)

    
    result = BenchmarkResult(
        sample_id=test_vcf.split(".")[0],
        snp_precision=snp_precision,
        snp_recall=snp_recall,
        snp_f1=snp_f1,
        indel_precision=indel_precision,
        indel_recall=indel_recall,
        indel_f1=indel_f1
    )
    return result

In [43]:
def extract_sample_metadata(analyte, metadata):
    """
    Extract sample metadata including median depth of coverage, input mass, and kit type.
    :param analyte: Analyte ID of the sample.
    :param metadata: DataFrame containing metadata information.
    :return: SampleMetadata containing sample metadata.
    """

    row = metadata.loc[int(analyte)]
    kit = row['kit']

    wgs_data_csv = pd.read_csv(f'{analyte}.CollectWgsMetrics.coverage_metrics', sep='\t', comment='#', nrows=1)
    depth_of_coverage = int(wgs_data_csv['MEDIAN_COVERAGE'])
    metadata = SampleMetadata(
        sample_id=analyte,
        median_depth_of_coverage=depth_of_coverage,
        kit=kit
    )
    return metadata

## Analysis on Taps and Unconverted

In [ ]:
input_bams = glob.glob("*.deduped.bam")
tvc_results = []
accuracy_results = []
metadata_results = []
for bam in input_bams:
    tvc_result = run_TVC(bam, REFERENCE_GENOME, cores=90)
    accuracy_results.append(extract_accuracy_metrics(BENCHMARK_VCF, tvc_result.vcf_path, BENCHMARK_REGIONS, REFERENCE_GENOME))
    metadata_results.append(extract_sample_metadata(os.path.basename(bam.split(".")[0]), METADATA))
    tvc_results.append(tvc_result)


In [ ]:
tvc_df = pd.DataFrame(tvc_results)
tvc_df = tvc_df.set_index('sample_id')
accuracy_df = pd.DataFrame(accuracy_results)
accuracy_df = accuracy_df.set_index('sample_id')
metadata_df = pd.DataFrame(metadata_results)
metadata_df = metadata_df.set_index('sample_id')

final_df = pd.concat([tvc_df, accuracy_df, metadata_df], axis=1)
final_df.to_csv("tvc_benchmarking_results.tsv", sep="\t")

## Results of AGBT run

In [56]:
final_df = final_df.sort_values(by='sample_id')

final_df

,runtime_minutes,memory_mb,cores_used,vcf_path,snp_precision,snp_recall,snp_f1,indel_precision,indel_recall,indel_f1,median_depth_of_coverage,kit
sample_id,,,,,,,,,,,,
207951,24.650737,7060.050781,90,207951.deduped.tvc.vcf,0.941527,0.896835,0.918638,0.615959,0.587822,0.601562,14,Unconverted
207952,23.537730,7040.777344,90,207952.deduped.tvc.vcf,0.940385,0.888163,0.913528,0.615445,0.578782,0.596551,13,Unconverted
207953,29.591450,7116.457031,90,207953.deduped.tvc.vcf,0.949850,0.925603,0.937570,0.684748,0.643357,0.663408,17,Unconverted
207954,31.936335,7154.558594,90,207954.deduped.tvc.vcf,0.951663,0.932178,0.941820,0.687224,0.650848,0.668541,18,Unconverted
207955,30.058229,7098.792969,90,207955.deduped.tvc.vcf,0.953963,0.928902,0.941266,0.755142,0.676487,0.713654,17,Unconverted
207956,31.625189,7114.699219,90,207956.deduped.tvc.vcf,0.955625,0.934946,0.945172,0.758445,0.684020,0.719312,18,Unconverted
207957,22.777119,7007.480469,90,207957.deduped.tvc.vcf,0.917645,0.853727,0.884533,0.592129,0.552927,0.571857,13,TAPS
207958,22.842824,6999.390625,90,207958.deduped.tvc.vcf,0.917335,0.855226,0.885192,0.590116,0.552749,0.570822,13,TAPS
207959,31.407134,7113.320312,90,207959.deduped.tvc.vcf,0.932088,0.904310,0.917989,0.642485,0.622593,0.632383,18,TAPS


In [53]:
demo_tvc_results = run_TVC("999999.bam", REFERENCE_GENOME, cores=90)
demo_happy_results = extract_accuracy_metrics(BENCHMARK_VCF, demo_tvc_results.vcf_path, BENCHMARK_REGIONS, REFERENCE_GENOME)
demo_metadata = extract_sample_metadata("999999", METADATA)

/tmp/ipykernel_1467351/2730702165.py:13: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  depth_of_coverage = int(wgs_data_csv['MEDIAN_COVERAGE'])


In [64]:
demo_tvc_df = pd.DataFrame([demo_tvc_results])
demo_happy_df = pd.DataFrame([demo_happy_results])
demo_metadata_df = pd.DataFrame([demo_metadata])

demo_final_df = pd.concat([demo_tvc_df, demo_happy_df, demo_metadata_df], axis=1)
demo_final_df.to_csv("demo_tvc_benchmarking_results.tsv", sep="\t")

## Below are results on the demo set

In [65]:
demo_final_df

,sample_id,runtime_minutes,memory_mb,cores_used,vcf_path,sample_id,snp_precision,snp_recall,snp_f1,indel_precision,indel_recall,indel_f1,sample_id,median_depth_of_coverage,kit
0,999999,62.076779,6997.175781,90,999999.tvc.vcf,999999,0.976836,0.943045,0.959643,0.716975,0.708737,0.712832,999999,40,TAPS
